In [1]:
import os
import sys
import torch

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
from graph_bridges.configs.graphs.config_sb import SBConfig
from graph_bridges.configs.graphs.config_sb import TrainerConfig
from graph_bridges.data.graph_dataloaders_config import EgoConfig,CommunityConfig,CommunitySmallConfig
from graph_bridges.models.backward_rates.backward_rate_config import BackRateMLPConfig
from graph_bridges.configs.graphs.config_sb import SBConfig, ParametrizedSamplerConfig, SteinSpinEstimatorConfig
from graph_bridges.models.backward_rates.backward_rate_config import GaussianTargetRateImageX0PredEMAConfig

In [3]:
from graph_bridges.models.generative_models.sb import SB

In [4]:
config = SBConfig(delete=True)

#config.data = EgoConfig(as_image=False, batch_size=32, full_adjacency=False)
#config.data = CommunityConfig(as_image=False, batch_size=32, full_adjacency=False)
config.data = EgoConfig(as_image=False, batch_size=32, full_adjacency=False)
#config.data = CommunitySmallConfig(as_image=False, batch_size=32, full_adjacency=False)
config.model = GaussianTargetRateImageX0PredEMAConfig(time_embed_dim=12, fix_logistic=False)

#config.model = BackRateMLPConfig(time_embed_dim=14,hidden_layer=150)
config.stein = SteinSpinEstimatorConfig(stein_sample_size=100)
config.sampler = ParametrizedSamplerConfig(num_steps=10)
config.optimizer = TrainerConfig(learning_rate=1e-3,
                                 num_epochs=200,
                                 save_metric_epochs=20,
                                 metrics=["graphs_plots",
                                          "histograms"])
config.align_configurations()
#read the model
device = torch.device("cpu")
sb = SB(config, device)

In [5]:
databatch = next(sb.data_dataloader.train().__iter__())
x_adj = databatch[0]

In [ ]:
class SpinsToBinaryTensor:

    def __call__(self, spins):
        binary_tensor = int( (1. + spins)*.5)
        return binary_tensor


In [7]:
from torchvision.transforms import Lambda

In [12]:
SpinsToBinaryTensor = Lambda(lambda x: ( (1. + x)*.5).float())

In [13]:
SpinsToBinaryTensor(x_adj)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.]])